In [128]:
from tracemalloc import stop
import numpy as np
import pandas as pd
import re
import seaborn as sns
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier

In [129]:
nltk.download('punkt')
nltk.download('stopwords')
sw=nltk.corpus.stopwords.words("english")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [130]:
!pip install emoji


[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [131]:
import emoji
import string
import re

In [188]:
df = pd.read_csv("UScomments.csv",error_bad_lines=False)

C:\Users\LENOVO\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3552: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 41589: expected 4 fields, saw 11\nSkipping line 51628: expected 4 fields, saw 7\nSkipping line 114465: expected 4 fields, saw 5\n'
b'Skipping line 142496: expected 4 fields, saw 8\nSkipping line 189732: expected 4 fields, saw 6\nSkipping line 245218: expected 4 fields, saw 7\n'
b'Skipping line 388430: expected 4 fields, saw 5\n'
C:\Users\LENOVO\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3552: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [189]:
df.head()

,video_id,comment_text,likes,replies
0,XpVt6Z1Gjjo,Logan Paul it's yo big day ‼️‼️‼️,4,0
1,XpVt6Z1Gjjo,I've been following you from the start of your...,3,0
2,XpVt6Z1Gjjo,Say hi to Kong and maverick for me,3,0
3,XpVt6Z1Gjjo,MY FAN . attendance,3,0
4,XpVt6Z1Gjjo,trending 😉,3,0


In [138]:
Usvideos = pd.read_csv(r"USvideos.csv",error_bad_lines=False)

C:\Users\LENOVO\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3552: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 2401: expected 11 fields, saw 21\nSkipping line 2800: expected 11 fields, saw 21\nSkipping line 5297: expected 11 fields, saw 12\nSkipping line 5299: expected 11 fields, saw 12\nSkipping line 5300: expected 11 fields, saw 12\nSkipping line 5301: expected 11 fields, saw 12\n'


In [139]:
Usvideos.head()

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09
2,cLdxuaxaQwc,My Response,PewDiePie,22,[none],5845909,576597,39774,170708,https://i.ytimg.com/vi/cLdxuaxaQwc/default.jpg,13.09
3,WYYvHb03Eog,Apple iPhone X first look,The Verge,28,apple iphone x hands on|Apple iPhone X|iPhone ...,2642103,24975,4542,12829,https://i.ytimg.com/vi/WYYvHb03Eog/default.jpg,13.09
4,sjlHnJvXdQs,iPhone X (parody),jacksfilms,23,jacksfilms|parody|parodies|iphone|iphone x|iph...,1168130,96666,568,6666,https://i.ytimg.com/vi/sjlHnJvXdQs/default.jpg,13.09


In [140]:
Usvideos.shape

(7992, 11)

In [190]:
df.shape

(691400, 4)

In [191]:
df.dropna(inplace=True)

In [192]:
df.isnull().sum()

video_id        0
comment_text    0
likes           0
replies         0
dtype: int64

In [193]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 691375 entries, 0 to 691399
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   video_id      691375 non-null  object
 1   comment_text  691375 non-null  object
 2   likes         691375 non-null  object
 3   replies       691375 non-null  object
dtypes: object(4)
memory usage: 26.4+ MB


In [194]:
df.drop(41587, inplace=True)

In [195]:
df = df.reset_index().drop('index',axis=1)

In [196]:
df.likes = df.likes.astype(int)
df.replies = df.replies.astype(int)

In [197]:
df.head()

,video_id,comment_text,likes,replies
0,XpVt6Z1Gjjo,Logan Paul it's yo big day ‼️‼️‼️,4,0
1,XpVt6Z1Gjjo,I've been following you from the start of your...,3,0
2,XpVt6Z1Gjjo,Say hi to Kong and maverick for me,3,0
3,XpVt6Z1Gjjo,MY FAN . attendance,3,0
4,XpVt6Z1Gjjo,trending 😉,3,0


# Prepocessing Task

In [255]:
df['comment_text'] = df['comment_text'].str.replace("[^a-zA-Z#]", " ")

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_7204\1720343950.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['comment_text'] = df['comment_text'].str.replace("[^a-zA-Z#]", " ")


In [263]:
df['comment_text'][45]

'this looks like great video after done would kind enough check youtube channel give some feedback maybe even subscribing'

In [199]:
df['comment_text'] = df['comment_text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

In [200]:
df['comment_text'] = df['comment_text'].apply(lambda x:x.lower())

In [201]:
tokenized_tweet = df['comment_text'].apply(lambda x: x.split())
tokenized_tweet.head()

0                                        [logan, paul]
1    [been, following, from, start, your, vine, cha...
2                                     [kong, maverick]
3                                         [attendance]
4                                           [trending]
Name: comment_text, dtype: object

# Lemmatization 

In [202]:
import nltk 
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [203]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [204]:
wnl = WordNetLemmatizer()

In [205]:
tokenized_tweet.apply(lambda x: [wnl.lemmatize(i) for i in x if i not in set(stopwords.words('english'))]) 
tokenized_tweet.head()

LookupError: 
**********************************************************************
  Resource [93mwordnet[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('wordnet')
  [0m
  Searched in:
    - 'C:\\Users\\LENOVO/nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - 'C:\\Users\\LENOVO\\anaconda3\\nltk_data'
    - 'C:\\Users\\LENOVO\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\LENOVO\\AppData\\Roaming\\nltk_data'
**********************************************************************


In [206]:
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])

In [207]:
df['comment_text'] = tokenized_tweet

In [264]:
df['comment_text']

0                                                logan paul
1         been following from start your vine channel ha...
2                                             kong maverick
3                                                attendance
4                                                  trending
                                ...                        
691369                                                     
691370                              aimerais viennes roanne
691371                                               mexico
691372                                                yeter
691373    kocham piosenk zakochana uszy moja colonna sonora
Name: comment_text, Length: 691374, dtype: object

In [208]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [209]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

In [210]:
df['Sentiment Scores'] = df['comment_text'].apply(lambda x:sia.polarity_scores(x)['compound'])

In [211]:
df.head()

,video_id,comment_text,likes,replies,Sentiment Scores
0,XpVt6Z1Gjjo,logan paul,4,0,0.0
1,XpVt6Z1Gjjo,been following from start your vine channel ha...,3,0,0.0
2,XpVt6Z1Gjjo,kong maverick,3,0,0.0
3,XpVt6Z1Gjjo,attendance,3,0,0.0
4,XpVt6Z1Gjjo,trending,3,0,0.0


# Classifying the Sentiment scores as Positive, Negative and Neutral

In [212]:
df['Sentiment'] = df['Sentiment Scores'].apply(lambda s : 'Positive' if s > 0 else ('Neutral' if s == 0 else 'Negative'))

In [213]:
df.head()

,video_id,comment_text,likes,replies,Sentiment Scores,Sentiment
0,XpVt6Z1Gjjo,logan paul,4,0,0.0,Neutral
1,XpVt6Z1Gjjo,been following from start your vine channel ha...,3,0,0.0,Neutral
2,XpVt6Z1Gjjo,kong maverick,3,0,0.0,Neutral
3,XpVt6Z1Gjjo,attendance,3,0,0.0,Neutral
4,XpVt6Z1Gjjo,trending,3,0,0.0,Neutral


In [214]:
df.Sentiment.value_counts()

Positive    305358
Neutral     260986
Negative    125030
Name: Sentiment, dtype: int64

In [215]:
df = df.rename(columns={'Sentiment': 'Target'})

In [216]:
df.head()

,video_id,comment_text,likes,replies,Sentiment Scores,Target
0,XpVt6Z1Gjjo,logan paul,4,0,0.0,Neutral
1,XpVt6Z1Gjjo,been following from start your vine channel ha...,3,0,0.0,Neutral
2,XpVt6Z1Gjjo,kong maverick,3,0,0.0,Neutral
3,XpVt6Z1Gjjo,attendance,3,0,0.0,Neutral
4,XpVt6Z1Gjjo,trending,3,0,0.0,Neutral


In [284]:
tfidf1=TfidfVectorizer(stop_words=sw,max_features=20)
def transform1(txt1):
    txt2=tfidf1.fit_transform(txt1)
#     txt2 = tfidf1.fit(txt1)
    return txt2.toarray()

In [285]:
x = transform1(df['comment_text'])
y=df["Target"]

In [286]:
# x_train1,x_test1,y_train1,y_test1=train_test_split(x,y,test_size=0.2,random_state=0)

In [287]:
model1=MultinomialNB()
model1.fit(x_train1,y_train1)

MultinomialNB()

# Testing Purpose For Transform Text Function

In [278]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# # Create an instance of the TF-IDF vectorizer
# vectorizer = TfidfVectorizer()

# # Fit the vectorizer with your data
# corpus = ["This is the first document.", "This document is the second document."]
# vectorizer.fit(corpus)

# # Transform your text using the fitted vectorizer
# text = "This is the third document."
# vectorized_text = vectorizer.transform([text])

# # Continue with further operations on vectorized_text


In [308]:
text="I hate you"

In [309]:
def new_transform(text):
    tmp_=[]
    for i in text:
        if i not in string.punctuation:
             tmp_.append(i)
    text = ''.join(tmp_)
    text = re.sub('[^A-Za-z ]+', '', text)
    text=text.strip()
    text = ' '.join([w for w in text.split() if len(w)>3])
    text = text.lower()
    # text = text.split()
    # text = [wnl.lemmatize(i) for i in text if i not in set(stopwords.words('english'))]
    # text = ' '.join(text)
    return text

In [310]:
new_transform(text)

'hate'

In [311]:
tfidf1.transform([new_transform(text)])

<1x20 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [312]:
new_str = new_transform(text)
tmp = tfidf1.transform([new_str])
pred = model1.predict(tmp.todense())
print(new_str)
print(pred[0])

hate
Positive


In [313]:
df['Target'].value_counts()

Positive    305358
Neutral     260986
Negative    125030
Name: Target, dtype: int64

In [114]:
# df.dropna(inplace=True)

In [115]:
!pip install textblob


[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [116]:
from textblob import TextBlob

In [117]:
polarity = []
for comment in df['comment_text']:
    try:
        polarity.append(TextBlob(comment).sentiment.polarity)
    except:
        polarity.append(0)

In [118]:
df['Polarity'] = polarity

In [119]:
x=[]
for i in df.Polarity:
    if i>=0.5:
        x.append('Positive')
    else:
        x.append('Negative')
#     else:
#         x.append('Neutral')
        

In [120]:
df.head()

,video_id,comment_text,likes,replies,Polarity
0,XpVt6Z1Gjjo,Logan Paul it's yo big day ‼️‼️‼️,4,0,0.0
1,XpVt6Z1Gjjo,I've been following you from the start of your...,3,0,0.0
2,XpVt6Z1Gjjo,Say hi to Kong and maverick for me,3,0,0.0
3,XpVt6Z1Gjjo,MY FAN . attendance,3,0,0.0
4,XpVt6Z1Gjjo,trending 😉,3,0,0.0


In [121]:
df['Target'] = x

In [122]:
df.head()

,video_id,comment_text,likes,replies,Polarity,Target
0,XpVt6Z1Gjjo,Logan Paul it's yo big day ‼️‼️‼️,4,0,0.0,Negative
1,XpVt6Z1Gjjo,I've been following you from the start of your...,3,0,0.0,Negative
2,XpVt6Z1Gjjo,Say hi to Kong and maverick for me,3,0,0.0,Negative
3,XpVt6Z1Gjjo,MY FAN . attendance,3,0,0.0,Negative
4,XpVt6Z1Gjjo,trending 😉,3,0,0.0,Negative


In [123]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score

In [124]:
def transform_text(text):
#     text1 = convert_emojis_to_word(text)
#     text.encode('utf-8')
    text1=text.lower()
    text1=nltk.word_tokenize(text1)
#     print(text1)
    y=[]
    for i in text1:
        if i.isalnum():
#         if i.islower() or i.encode('utf-8'):
#         if i.islower():
            y.append(i)
    text1=y[:]
#     print(text1)
    y.clear()
    for i in text1:
        if i not in stopwords.words('english') and i not in string.punctuation:
            y.append(i)
#         if i in emoji.distinct_emoji_list(i):
#             y.append(i)
    text1=y[:]
#     print(text1)
    y.clear()
    ps=PorterStemmer()
    for i in text1:
        y.append(ps.stem(i))
#     print(text1)
    return " ".join(y)

In [125]:
text='Hello I am Happy Now!! and what about u'

In [126]:
transform_text(text)

'hello happi u'

In [96]:
tfidf1=TfidfVectorizer(stop_words=sw,max_features=20)
def transform1(txt1):
    txt2=tfidf1.fit_transform(txt1)
    return txt2.toarray()


In [97]:
x=transform1(df["comment_text"])
y=df["Target"]


In [98]:
# x_train1,x_test1,y_train1,y_test1=train_test_split(x,y,test_size=0.2,random_state=0)

In [99]:
model1=MultinomialNB()
model1.fit(x_train1,y_train1)

MultinomialNB()

In [63]:
str = 'I hate you'

In [82]:
new_str = transform_text(str)
new_str

'1 trend fuck ye'

In [83]:
ex_tfidf = tfidf1.transform([new_str])

In [84]:
pred = model1.predict(ex_tfidf)

In [85]:
pred[0]

'Positive'

In [68]:
negative_df = df[df['Target']=='Negative']

In [73]:
negative_df.head(20)

,video_id,comment_text,likes,replies,Polarity,Target
9,XpVt6Z1Gjjo,You and your shit brother may have single hand...,0,0,-0.135714,Negative
12,XpVt6Z1Gjjo,Honestly Evan is so annoying. Like its not fun...,0,0,-0.023333,Negative
16,XpVt6Z1Gjjo,Ayyyyoooo Logang what up . This was a hard vl...,1,0,-0.291667,Negative
19,XpVt6Z1Gjjo,Made a lot of people hate youtube - GJ,0,0,-0.800000,Negative
28,XpVt6Z1Gjjo,Evan is a horrible human being he also looks s...,0,0,-0.050000,Negative
34,XpVt6Z1Gjjo,Gotta love Youtube for giving morons the abili...,0,0,-0.306250,Negative
36,XpVt6Z1Gjjo,Can the Pauls please stop saying they're the f...,1,0,-0.364583,Negative
42,XpVt6Z1Gjjo,Not a hater but fuck you,0,0,-0.400000,Negative
43,XpVt6Z1Gjjo,Where is the other dog,0,0,-0.125000,Negative
44,XpVt6Z1Gjjo,I go out of my way to dislike every single one...,0,0,-0.071429,Negative


In [80]:
str = negative_df['comment_text'].loc[91]

In [81]:
TextBlob(str).sentiment.polarity

-0.5

In [127]:
df['Target'].value_counts()

Negative    562901
Positive    128474
Name: Target, dtype: int64